In [ ]:
import tensorflow as tf
numGPUs = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", numGPUs)

In [ ]:
import random
import pandas
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D,Dropout,Activation,MaxPooling2D,AveragePooling2D,GlobalAveragePooling2D
from keras import backend as K
import talos
import os
import math
import threading
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import collections
from pathlib import Path
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
numClients = 20
num_classes = 10
dataset_name = 'svhn_cropped'
experiment_name = 'svhn'
Path(experiment_name+"_res/res"+str(numClients)).mkdir(parents=True, exist_ok=True)

In [ ]:
# define the grid search parameters
def get_params_for_X_clients(x):    
    act_fn = ["relu"]
    
    batch_size = [64]#[32,16,8,4,2]#[32,64,128]#[8,16,32]#[256,128,64,32]
    epochs = [125]#[75,125]
    learn_rate = [0.1]#[0.1,0.15,0.2,0.25,0.3]#[0.08, 0.1, 0.2, 0.3]#[0.001,0.01, 0.1]
    momentum = [0.9]#[0.7,0.8,0.85,0.9,0.95]#[0.88, 0.89, 0.90, 0.91, 0.92, 0.93, 0.94]#[0.88, 0.9, 0.92, 0.93, 0.95]#[0.0, 0.3, 0.6, 0.9]
    
    
    param_grid = dict(learn_rate=learn_rate, batch_size=batch_size, epochs=epochs, act_fn=act_fn, momentum=momentum)
    
    return param_grid

In [ ]:
def get_model(params):
    model = Sequential()
    model.add(Conv2D(6, (5,5), activation=params['act_fn'], input_shape=input_shape,name="1"))
    model.add(AveragePooling2D((2,2),name="2"))
    model.add(Conv2D(16, (5,5), activation=params['act_fn'],name="3"))
    model.add(AveragePooling2D((2,2),name="4"))
    model.add(Flatten(name="5"))
    model.add(Dense(120, activation=params['act_fn'],name="6"))
    model.add(Dense(84, activation=params['act_fn'],name="7"))
    model.add(Dense(num_classes, activation='softmax',name="f"))
    
    return model

In [ ]:
# Model / data parameters
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = tfds.as_numpy(tfds.load(dataset_name,
                                                               split=['train','test'],
                                                               batch_size=-1,
                                                               as_supervised=True))

# Scale images to the [0, 1] range
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = x_train / 255
x_test = x_test / 255
# Make sure images have shape (28, 28, 1)
print(x_train.shape)
print(x_test.shape)
#x_train = np.expand_dims(x_train, -1)
#x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")
y_test_uncat = y_test

#to use with mean sq error
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#shuffle both train and test once (to average between runs later on..)
shuffler = np.random.permutation(len(x_train))
x_train = x_train[shuffler]
y_train = y_train[shuffler]
shuffler = np.random.permutation(len(x_test))
x_test = x_test[shuffler]
y_test = y_test[shuffler]

sample_height = x_train[0].shape[0]
sample_width = x_train[0].shape[1]
sample_channels = x_train[0].shape[2]


input_shape = (sample_height, sample_width, sample_channels)

def randomSplitClientsData(data,labels,numParties):
    numSamplesPerClient = int(data.shape[0]/numParties)
    print(numSamplesPerClient)
    clientsData = np.zeros((numParties,int(numSamplesPerClient),sample_height,sample_width,sample_channels))
    clientsDataLabels = np.zeros((numParties,int(numSamplesPerClient),num_classes))
    #print(numSamplesPerClient)
    ind = 0
    for i in range(numParties):
        clientsData[i] = data[ind:ind+numSamplesPerClient]
        clientsDataLabels[i]=labels[ind:ind+numSamplesPerClient]
        ind = ind+numSamplesPerClient
    return clientsData, clientsDataLabels

def prepare_data_for_X_clients(numClients):
    clientsData, clientsDataLabels = randomSplitClientsData(x_train, y_train, numClients)
    return clientsData, clientsDataLabels

In [ ]:
clientsData, clientsDataLabels = prepare_data_for_X_clients(numClients)

In [ ]:
f = plt.figure(figsize=(12,7))
for i in range(numClients):
    client_labels = clientsDataLabels[i]
    plot_data = collections.defaultdict(list)
        
    for l in client_labels:
        plot_data[np.argmax(l)].append(np.argmax(l))
    plt.subplot(int(math.ceil(numClients/3)), 3, i+1)
    plt.title('Client {}'.format(i))
    for j in range(numClients):
        plt.hist(
            plot_data[j],
            density=False,
            bins=list(range(num_classes+1)))

In [ ]:
r = np.random.randint(0, len(x_train))
plt.imshow(x_train[r])
_ = plt.show()
print(np.argmax(y_train[r]))

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history, params):
    print('##########################################################')
    print(params)
    # plot loss
    plt.subplot(211)
    plt.title('MSE')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    #filename = sys.argv[0].split('/')[-1]
    #pyplot.savefig(filename + '_plot.png')
    #pyplot.close()
    plt.show()
    print('##########################################################')

In [ ]:
hist = []
hist_params = []

def experiment(x_train, y_train, x_val, y_val, params):
        
    
    optimizer = SGD(learning_rate=params['learn_rate'], momentum=params['momentum'], nesterov=False, name='SGD')
    
    model = get_model(params)
    model.compile(optimizer=optimizer,
                  loss="mean_squared_error",
                  metrics=['accuracy', tf.keras.metrics.Recall(),tf.keras.metrics.Precision()],
                  run_eagerly=False)

    early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  min_delta=0.01,
                                                  patience=5)
    history = model.fit(x=x_train,
                    y=y_train,
                    epochs=params['epochs'],
                    batch_size=params['batch_size'],
                    callbacks=[early_stop],
                    validation_data=(x_val, y_val),
                    verbose=0)

    hist.append(history)
    hist_params.append(params)
        
    return history, model

In [ ]:
def grid_search_for_X_clients(numClients, clientsData, clientsDataLabels, param_grid):

    scan_res = np.zeros(numClients, dtype=object)
    
    
    def client_gridsearch(i):
        #Distribute load accross GPUs
        with tf.device('/GPU:'+str(i%numGPUs)):
        
            print('training in client ', i)
            scan_results = talos.Scan(x=clientsData[i],
                                      y=clientsDataLabels[i],
                                      params=param_grid,
                                      model=experiment,
                                      experiment_name=experiment_name)
            scan_res[i]=scan_results
        
    # Batch multithreading
    n_batch = int(math.ceil(float(numClients)/(numGPUs)))
    print(n_batch)
    remaining_clients = numClients
    for i in range(n_batch):
        threads = list()

        for j in range(min(numGPUs, remaining_clients)):
            t = threading.Thread(target=client_gridsearch, args=(i*(numGPUs)+j,))
            threads.append(t)
            t.start()

        for _,t in enumerate(threads):
            remaining_clients -= 1
            t.join()
    
    return scan_res

In [ ]:
big_res = []
hist = []
hist_params = []
    
param_grid = get_params_for_X_clients(numClients)

res = grid_search_for_X_clients(numClients, clientsData, clientsDataLabels, param_grid)

big_res=res

In [ ]:
## Sort dataframes
sorted_data = []
big_res = [r for r in big_res if not r == 0]

for _,df in enumerate(big_res):
    sorted_data.append(df.data.sort_values(by='val_accuracy',ascending=False).head())

## Write dataframes to files
for i,df in enumerate(sorted_data):
    df.to_csv(experiment_name+"_res/res"+str(numClients)+"/res"+str(numClients)+"_client_"+str(i)+".csv")

for i in range(len(sorted_data)):
    with pandas.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(sorted_data[i])

In [ ]:
avg_params = np.zeros(4, dtype=float) #[lr, batchsize, epochs, momentum]

for _,client_data in enumerate(sorted_data):
    avg_params[0] += client_data.head(1)['learn_rate'].item()
    avg_params[1] += client_data.head(1)['batch_size'].item()
    avg_params[2] += client_data.head(1)['round_epochs'].item()
    avg_params[3] += client_data.head(1)['momentum'].item()
    
avg_params = avg_params / len(sorted_data)

print("Avg lr:", avg_params[0], "Avg batchsize:", int(math.ceil(avg_params[1])), "Avg epochs:", int(math.ceil(avg_params[2])), "Avg momentum:",avg_params[3])    

In [ ]:
summarize_diagnostics(hist[0], hist_params[0])

In [ ]:
def test_metrics_for_X_clients(numClients, clientsData, clientsDataLabels, avg_test_params):

    metrics_res = np.zeros(numClients, dtype=object)
    
    
    def client_test_metrics(i):
        #Distribute load accross GPUs
        with tf.device('/GPU:'+str(i%numGPUs)):

            print('training in client ', i)
            optimizer = SGD(learning_rate=avg_test_params['learn_rate'], momentum=avg_test_params['momentum'], nesterov=False, name='SGD')


            model = get_model(avg_test_params)
            model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

            model.fit(x=clientsData[i],
                        y=clientsDataLabels[i],
                        epochs=avg_test_params['epochs'],
                        batch_size=avg_test_params['batch_size'],
                        verbose=1)
            print('evaluation in client ', i)
            metrics = model.evaluate(x_test, y_test)
            metrics_res[i] = metrics
            
        
    # Batch multithreading
    n_batch = int(math.ceil(float(numClients)/(numGPUs)))
    print(n_batch)
    remaining_clients = numClients
    for i in range(n_batch):
        threads = list()

        for j in range(min(numGPUs, remaining_clients)):
            t = threading.Thread(target=client_test_metrics, args=(i*(numGPUs)+j,))
            threads.append(t)
            t.start()

        for _,t in enumerate(threads):
            remaining_clients -= 1
            t.join()
    
    return metrics_res

In [ ]:
####### Retrain each client to run the test set and get best metrics
avg_test_params = dict(learn_rate=avg_params[0], batch_size=int(math.ceil(avg_params[1])), epochs=int(math.ceil(avg_params[2])), act_fn='relu', momentum=avg_params[3])
avg_test_res = []

clientsData, clientsDataLabels = prepare_data_for_X_clients(numClients)

res = test_metrics_for_X_clients(numClients, clientsData, clientsDataLabels, avg_test_params)

avg_test_res=res

In [ ]:
best_val_acc = 0.0
best_precision = 0.0
best_recall = 0.0

avg_test_res = [r for r in avg_test_res if not r == 0]

for _,client_metrics in enumerate(avg_test_res):
    if client_metrics[1] >= best_val_acc:
        best_val_acc = client_metrics[1]
        best_precision = client_metrics[2]
        best_recall = client_metrics[3]
    
print("Best val_acc: ", best_val_acc)
print("Best precision: ", best_precision)
print("Best recall: ", best_recall)

In [ ]:
####### Activation functions optimization

In [ ]:
act_fn = ["relu", "sigmoid", "tanh"]
act_fn_params = dict(learn_rate=[avg_params[0]], batch_size=[int(math.ceil(avg_params[1]))], epochs=[int(math.ceil(avg_params[2]))],momentum=[avg_params[3]], act_fn=act_fn)
act_fn_params

In [ ]:
act_fn_big_res = []
hist = []
hist_params = []
    
clientsData, clientsDataLabels = prepare_data_for_X_clients(numClients)
param_grid = get_params_for_X_clients(numClients)

res = grid_search_for_X_clients(numClients, clientsData, clientsDataLabels, act_fn_params)

act_big_res=res

In [ ]:
## Sort dataframes
act_big_res = [r for r in act_big_res if not r == 0]
sorted_data_activation = []
for _,df in enumerate(act_big_res):
    sorted_data_activation.append(df.data.sort_values(by='val_accuracy',ascending=False))

for i in range(len(sorted_data_activation)):
    with pandas.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(sorted_data_activation[i])

In [ ]:
## Get best activation function
act_fn_count = np.zeros(len(act_fn), dtype=float)

for _,client_data in enumerate(sorted_data_activation):
    for i,fn in enumerate(act_fn):
        if client_data.head(1)['act_fn'].item() == fn:
            act_fn_count[i] += 1
            
best_act_fn = act_fn[np.argmax(act_fn_count)]
print("Best activation function :", best_act_fn)